In [17]:
import numpy as np
N=100
kar = np.arange(N)*2

In [38]:
def generate_data (sig = 1e-3, A=0.1, phi_ofs=np.random.uniform(0,2*np.pi)):
    n = np.random.normal (0, 1/np.sqrt(2),N)+1j*np.random.normal(0,1/np.sqrt(2),N)
    s = A*np.exp(1j*(phi_ofs+kar*sig))
    return s+n

def estimate_phi (data,res=False,fds=False,array=False):
    sum0 = np.sum(data)
    sum1 = data*kar*1j
    sum2 = np.sum(-data*kar**2)    
    fd_arr = np.real(sum1)
    
    sum1 = np.sum(sum1)
    FD = np.real(sum0*np.conj(sum1))
    SD = np.real(sum2*np.conj(sum0)+sum1*np.conj(sum1))
    
    if fds:
        return FD
    elif res:
        return FD/SD
    elif array:
        return fd_arr
    else:
        return SD



def get_error (sig, A, phi_ofs=np.random.uniform(0,2*np.pi), Ngo=100000):
    res = [estimate_phi(generate_data(sig, A),res=True) for i in range(Ngo)]
    res = np.array(res)
    mean = res.mean()
    std = np.sqrt((res**2).mean()-sig**2)
    pred = var_phi(sig, A, Ngo)
    
    return mean, std, pred

In [39]:
def var_phi(sig, A, Ngo):
    fds = [estimate_phi(generate_data(sig, A),fds=True) for i in range(Ngo)]
    fds = np.array(fds)
    var_fd = np.var(fds)
    sd = estimate_phi(generate_data(sig, A))
    print(sd)
    print(var_fd)
    
    return var_fd/(sd**2)

In [40]:
get_error(1e-3, 5.0,0, 100000)

-844908474.0293612
41648917261.94592


(0.0010067865534302007, 0.00027587674652768135, 5.834241253855938e-08)

In [41]:
def best_drift(Ngo,sig,A):
    num = []
    denom = []
    for i in range(Ngo):
        data = generate_data(sig, A)
        fd = estimate_phi(data,array=True)
        sd = estimate_phi(data)  
        phi = estimate_phi(data,res=True)
        num.append(phi/(np.var(fd)/(sd**2)))
        denom.append(1/(np.var(fd)/(sd**2)))
        
    weight = np.sum(num)
    scale = np.sum(denom)
    return weight/scale

In [42]:
best_drift(100000,1e-3,5.0)

0.0010160110688435702